### Imports

In [ ]:
# Standard library imports
import os
import sys
import importlib
import numpy as np
import random
import cv2
from tqdm import tqdm

# Add project-specific paths
sys.path.append(os.path.join(os.getcwd(), 'common'))

# Import custom modules
import utility
import embedding
import detection
import grt

# Reload module for updates
importlib.reload(grt)

### Embedding

In [2]:
# Randomly select 10 images
img_folder = 'sample_imgs'
img_files =  [f for f in os.listdir(img_folder) if f.endswith(('.bmp'))]
img_files = random.sample(img_files, 10)
images = []
for file in img_files:
    img_path = os.path.join(img_folder, file)
    images.append(cv2.imread(img_path, 0))
images = np.array(images) # optional

mark = np.load('ammhackati.npy')

alpha = 0.5

watermarked = []
for img in images:
    watermarked.append(embedding.embedding(img, mark, alpha))

### Global attacks

In [ ]:
progress_bar = tqdm(watermarked, desc = "Applying blur on 10 images")
history = []
successful_attacks = []
total = 0
succ = 0
max = 0

for index, wm in enumerate(progress_bar):
    for param in np.arange(0, 5, 0.2):
        attacked, attack_name, usd = grt.blur_gauss(wm, param)
        detected = detection.detection(images[index], wm, attacked, alpha, 2)
        wpsnr_attacked = utility.wpsnr(wm, attacked)
        log = {"image" : img_files[index],
                "attack" : attack_name,
                "param" : usd,
                "wpsnr" : wpsnr_attacked,
                "detection" : detected}
        history.append(log)
        if detected == 0:
            total += wpsnr_attacked
            succ +=1
            successful_attacks.append(log)
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            break

if succ > 0:                
    print("Average wpsnr of successful attacks: ", (total/succ))
    print("Max wpsnr of successful attacks: ", max) 
# for h in history:
#     print(h)   
for s in successful_attacks:
    print(s) 

In [ ]:
progress_bar = tqdm(watermarked, desc = "Applying blur on 10 images")
history = []
successful_attacks = []
total = 0
succ = 0
max = 0

for index, wm in enumerate(progress_bar):
    for param in np.arange(1, 15, 2):
        attacked, attack_name, usd = grt.blur_median(wm, param)
        detected = detection.detection(images[index], wm, attacked, alpha, 2)
        wpsnr_attacked = utility.wpsnr(wm, attacked)
        log = {"image" : img_files[index],
                "attack" : attack_name,
                "param" : usd,
                "wpsnr" : wpsnr_attacked,
                "detection" : detected}
        history.append(log)
        if detected == 0:
            total += wpsnr_attacked
            succ +=1
            successful_attacks.append(log)
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            break

if succ > 0:                
    print("Average wpsnr of successful attacks: ", (total/succ))
    print("Max wpsnr of successful attacks: ", max) 
# for h in history:
#     print(h)   
for s in successful_attacks:
    print(s) 